In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

from IPython.display import Image, display
from typing import Union

%matplotlib inline

In [ ]:
model_path = "ResnetV2_50.pb"

In [ ]:
# Load the graph from the .pb file
with tf.io.gfile.GFile(model_path, 'rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())

In [ ]:
with tf.compat.v1.Session() as sess:
    # Set the graph as the default graph
    tf.compat.v1.import_graph_def(graph_def, name='')
    graph = sess.graph

In [ ]:
def get_tensor(tensor_name: str) -> np.ndarray:
    with tf.compat.v1.Session() as sess:
        # Set the graph as the default graph
        tf.compat.v1.import_graph_def(graph_def, name='')

        # Get input and output tensors
        input_tensor = sess.graph.get_tensor_by_name("input:0")
        output_tensor = sess.graph.get_tensor_by_name(tensor_name)

        # Perform inference
        input_data = np.random.rand(1, 224, 224, 3)
        return sess.run(output_tensor, feed_dict={input_tensor: input_data})

In [ ]:
def get_images(tensor_name: str, feature_id: int) -> Image:
    return Image(url=f"https://openaipublic.blob.core.windows.net/microscopeprod/2020-07-25/2020-07-25/resnetv2_50_slim/lucid.dataset_examples/_dataset_examples/dataset%3Dimagenet%26op%3D{tensor_name.replace('/', '%252F')}%253A0/channel_{feature_id}_40.png")

In [ ]:
def to_op(tensor: Union[tf.Tensor, tf.Operation]) -> tf.Operation:
    if isinstance(tensor, tf.Tensor):
        return tensor.op
    return tensor

In [ ]:
def go_backwards(layer: Union[tf.Operation, str], num_layers: int=1) -> tf.Tensor:
    if isinstance(layer, str):
        layer = graph.get_operation_by_name(layer)
    previous_layer = layer
    for _ in range(num_layers):
        previous_layer = list(to_op(previous_layer).inputs)[0]
        print(previous_layer)
    return to_op(previous_layer)


In [ ]:
class_id = 2 # goldfish

In [ ]:
graph.get_operations()[-10:]

In [ ]:
logit_layer = "resnet_v2_50/logits/BiasAdd"
list(graph.get_operation_by_name(logit_layer).inputs) 

In [ ]:
weights_output, bias_layer = graph.get_operation_by_name(logit_layer).inputs

In [ ]:
bias = get_tensor(bias_layer.name)
plt.hist(bias, bins=100)
bias[class_id]

In [ ]:
weights_layer = graph.get_operation_by_name(weights_output.name.split(":")[0]).inputs[1]
weights_layer

In [ ]:
weight_matrix = get_tensor(weights_layer.name)
weight_matrix.shape

In [ ]:
relevant_weights = weight_matrix[0, 0, :, class_id]
ordering = (-relevant_weights).argsort()
relevant_weights.min(), relevant_weights.max()

In [ ]:
plt.hist(relevant_weights, bins=100)
None

In [ ]:
important_features = ordering[:5]
important_features

In [ ]:
relevant_weights[important_features]

In [ ]:
main_feature = important_features[0]
main_feature

In [ ]:
previous_layer = logit_layer
batchnorm_layer = go_backwards(previous_layer, 4)

In [ ]:
batchnorm_inputs = list(batchnorm_layer.inputs)
batchnorm_inputs

In [ ]:
previous_layer, *params = [get_tensor(layer.name) for layer in batchnorm_inputs]

In [ ]:
# create matplotlib figure with 4 subplots with histograms of each of the 4 parameters
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
axs = axs.flatten()
for i, param in enumerate(params):
    axs[i].hist(param.flatten(), bins=100)
    axs[i].set_title(batchnorm_inputs[i+1].name)

In [ ]:
gamma, beta, mean, variance = np.stack(params, axis=0)[:,main_feature]
gamma, beta, mean, variance

In [ ]:
f"f(x) = {gamma:.2f} * (x {-mean:+.2f})/{variance:.2f} {beta:+.2f}"

In [ ]:
gammas, betas, means, variances = np.stack(params, axis=0)
expected_features = (np.ones_like(gammas) - means) / variances + betas
plt.hist(expected_features, bins=100)
None

In [ ]:
expected_inputs =(np.ones_like(gammas) - means) / variances + betas
expected_inputs = -betas * variances + means
plt.hist(expected_inputs, bins=100)
None

In [ ]:
feature_layer = batchnorm_inputs[0].name.split(":")[0]
for i in range(5):
    display(get_images(feature_layer, important_features[i]))

In [ ]:
display(get_images(feature_layer, ordering[-1]))

In [ ]:
conv_out = batchnorm_inputs[0]
conv_out

In [ ]:
conv_inputs = list(to_op(conv_out).inputs)
conv_inputs

In [ ]:
previous_unit, bias = conv_inputs
conv_inputs = list(go_backwards(bias, 1).inputs)
conv_inputs

In [ ]:
bias_values = get_tensor(bias.name).flatten()
plt.hist(bias_values, bins=100)
bias_values[main_feature]

In [ ]:
_, conv_weights = conv_inputs
conv_weights = get_tensor(conv_weights.name)
conv_weights.shape

In [ ]:
relevant_weights = conv_weights[0, 0, :, main_feature]
plt.hist(relevant_weights, bins=100)
ordering = (-relevant_weights).argsort()

In [ ]:
important_features = ordering[:5]
relevant_weights[important_features]

In [ ]:
conv_inputs = list(go_backwards(conv_out, 2).inputs)
conv_inputs

In [ ]:
skip, add = conv_inputs